In [1]:
import findspark
findspark.init("/home/ubuntu/spark-2.1.1-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('BDAS-Linda').getOrCreate()

In [2]:
# import datasets
filename1 = "./Maori-Language-Learning-Student-Numbers-by-Ethnicity-2004-2019.csv"
filename2 = "./Subnational-ethnic-population-projections-2013base.csv"

MLLSN=spark.read.load(filename1, format="csv", header="true")
POP=spark.read.load(filename2, format="csv", header="true")

In [3]:
MLLSN.show()
POP.show()


+-------------------+------------------+------------------+-------------------+-----------------------------+---------------------------------------+------------------------------------------+----------+---------------------------------+--------------------+----------------------------------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------+------------------------+-------------------+------------------+-----------------+------------------------+------------------------+-----------------------------+------------------------------+-------------------------------------+------------------------+------------------------+--------------------+
|Students (? Values)|Year: As at 1 July|Student: Ethnicity|Student: Year level|Student: Year level (Grouped)|Student: Maori Language Immersion Level|Student: M?ori Language in Education level|School: ID|School: Maori Language Descriptor|      School: Medium|School: Highest Maori Lang

+----------------------------+---------------+-------------+--------+----------+----------+--------------+---------+-----------+-----------+--------------+---------+-----------+-----------+----------------+-----------+-------------+-------------+--------------+---------+-----------+-----------+
|TA with Auckland Local Board|Year at 30 June|Euro-all-ages|Euro-5-9|Euro-10-14|Euro-15-19|Maori-all-ages|Maori-5-9|Maori-10-14|Maori-15-19|Asian-all-ages|Asian-5-9|Asian-10-14|Asian-15-19|Pacific-all-ages|Pacific-5-9|Pacific-10-14|Pacific-15-19|Total-all-ages|Total-5-9|Total-10-14|Total-15-19|
+----------------------------+---------------+-------------+--------+----------+----------+--------------+---------+-----------+-----------+--------------+---------+-----------+-----------+----------------+-----------+-------------+-------------+--------------+---------+-----------+-----------+
|          Far North district|           2013|        40800|    2750|      2770|      2420|         27100|     2